Unduh data korpus Wikipedia Bahasa Indonesia melalui link berikut https://dumps.wikimedia.org/idwiki/latest/. Cari file dengan nama “idwiki-latest-pages-articles.xml.bz2”. Data korpus setiap waktunya mengalami peningkatan. Sekitar agustus 2019 terdapat 406855 kosa kata dengan ukuran 512 MB.

Data korpus tersebut masih menggunakan format XML sehingga perlu di olah ke dalam bentuk teks. library Gensim sudah menyediakan teknik pra poses ini. Untuk mengkonversi format tersebut dapat menggunakan kode berikut:

In [2]:
from __future__ import print_function
 
# Ignore warnings dari gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "idwiki-latest-pages-articles.xml.bz2"
namaFileOutput = "wiki.id.text"
 
space = " "
i = 0
 
# Write file ke variabel namaFileOutput encoder utf-8
output = open(namaFileOutput, 'w', encoding='utf-8')
 
# lower=False: huruf kecil dan besar dibedakan
wiki = WikiCorpus(namaFileInput, lemmatize=None, dictionary={}, lower=False)
for text in wiki.get_texts():
    output.write(' '.join(text) + '\n')
    i = i + 1
    if i % 10000 == 0:
        logger.info("Saved " + str(i) + " articles")
 
output.close()
logger.info("Finished Saved " + str(i) + " articles")

2023-10-30 11:28:24,940: INFO: running c:\Users\Naufal_Rz19\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"55caff74-20eb-43ca-99bd-ccb94398ef80" --shell=9007 --transport="tcp" --iopub=9009 --f=c:\Users\Naufal_Rz19\AppData\Roaming\jupyter\runtime\kernel-v2-8816JeV4t52J4oUh.json
2023-10-30 11:32:53,546: INFO: Saved 10000 articles
2023-10-30 11:35:30,640: INFO: Saved 20000 articles
2023-10-30 11:37:41,448: INFO: Saved 30000 articles
2023-10-30 11:39:23,776: INFO: Saved 40000 articles
2023-10-30 11:41:27,087: INFO: Saved 50000 articles
2023-10-30 11:43:23,151: INFO: Saved 60000 articles
2023-10-30 11:45:04,744: INFO: Saved 70000 articles
2023-10-30 11:46:48,147: INFO: Saved 80000 articles
2023-10-30 11:48:39,550: INFO: Saved 90000 articles
2023-10-30 11:50:06,216: INFO: Saved 100000 articles
2023-10-30 11:51:35,289: INFO: Saved 110000 articles
2023-1

Dari kode di atas akan menghasilkan output file dari rangkaian beberapa artikel, di mana satu barisnya mewakili satu artikel.

Setelah data berhasil terkonversi, selanjutnya dilakukan training Word2Vec menggunakan kode berikut:

In [3]:
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "wiki.id.text"
namaFileOutput = "w2vec_wiki_id300_0.txt"
 
 
# size 300 = 300 dimensi vektor, window 10 = 10 pengaruh kata disekitarnya, min_count 5 = kata-kata yang muncul < 5 kali akan dikeluarkan dari kosakata dan diabaikan selama pelatihan, sg 0 = cbow / sg 1 = skip gram 
model = Word2Vec(LineSentence(namaFileInput), vector_size=300, window=10, min_count=5, sg=0, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.wv.save_word2vec_format(namaFileOutput, binary=False)

2023-10-30 12:34:31,062: INFO: running c:\Users\Naufal_Rz19\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"55caff74-20eb-43ca-99bd-ccb94398ef80" --shell=9007 --transport="tcp" --iopub=9009 --f=c:\Users\Naufal_Rz19\AppData\Roaming\jupyter\runtime\kernel-v2-8816JeV4t52J4oUh.json
2023-10-30 12:34:31,277: INFO: collecting all words and their counts
2023-10-30 12:34:34,005: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-10-30 12:35:12,071: INFO: PROGRESS: at sentence #10000, processed 10738098 words, keeping 353874 word types
2023-10-30 12:35:38,834: INFO: PROGRESS: at sentence #20000, processed 17334952 words, keeping 490368 word types
2023-10-30 12:36:14,029: INFO: PROGRESS: at sentence #30000, processed 22148080 words, keeping 591133 word types
2023-10-30 12:36:28,157: INFO: PROGRESS: at sentence #40000, processed 2654

Terdapat beberapa parameter diantaranya:

<b>vector_size</b>: Ukuran dimensi vector yang mewakili setiap token atau kata. Jika memiliki data yang terbatas, nilai size yang digunakan sebaiknya lebih kecil karena akan mempengaruhi kata unik di sekitarnya. Namun, jika memiliki banyak data maka dapat bereksperimen dengan berbagai ukuran.

<b>window</b>: Jarak maksimum antara kata target dengan kata di sekitarnya. Jika nilai window yang digunakan besar, maka terdapat banyak kata terkait disekitarnya (baik di posisi kiri dan kanan dari kata konteks). Secara teori, jika window lebih kecil akan memberikan istilah yang lebih spesifik terhadap suatu konteks kata.

<b>min_count</b>: Frekuensi minimal jumlah kata. Model akan mengabaikan kata-kata yang tidak memenuhi nilai min_count. Kata yang jarang muncul biasanya tidak terlalu penting, jadi lebih baik untuk dihilangkan. Parameter ini mungkin lebih berpengaruh pada effisiensi penggunaan memory dan ukuran file model.

<b>sg</b>: 0 untuk arsitektur Word2Vec CBOW dan 1 untuk arsitektur Word2Vec Skip-gram.

<b>workers</b>: Berapa banyak threads yang digunakan untuk melakukan multiprocessing.

Model yang sudah disimpan dapat dilakukan skenario penggunaannya sebagaimana pada kode di bawah ini untuk melihat kedekatan hubungan antara kata.

In [4]:
import gensim
# import gensim.models.keyedvectors as word2vec
# model = gensim.models.Word2Vec.load(namaFileModel)
# model = word2vec.KeyedVectors.load_word2vec_format(namaFileModel, binary=True)

namaFileModel = "w2vec_wiki_id300_0.txt"

sim = model.wv.most_similar("Jakarta")
print("10 kata terdekat dari Jakarta:{}".format(sim))
sim = model.wv.most_similar("Bandung")
print("10 kata terdekat dari Bandung:{}".format(sim))
 
sim = model.wv.similarity("Yogyakarta", "Surakarta")
print("Kedekatan Yogyakarta-Surakarta: {}".format(sim))
sim = model.wv.similarity("Yogyakarta", "Semarang")
print("Kedekatan Yogyakarta-Semarang: {}".format(sim))
 
sim = model.wv.most_similar_cosmul(positive=['minuman', 'rendang'], negative=['makanan'])
print("makanan-rendang, minuman-?: {}".format(sim))
sim = model.wv.most_similar_cosmul(positive=['mobil', 'honda'], negative=['motor'])
print("motor-honda, mobil-?: {}".format(sim))

10 kata terdekat dari Jakarta:[('Warteg', 0.6368240714073181), ('Kemang', 0.6181235313415527), ('Surabaya', 0.5992199778556824), ('Cilandak', 0.5843774676322937), ('Kemayoran', 0.5826131701469421), ('Palmerah', 0.5699325799942017), ('Jagakarsa', 0.5650554299354553), ('Tebet', 0.5629286766052246), ('Senayan', 0.5624886751174927), ('Monas', 0.5526578426361084)]
10 kata terdekat dari Bandung:[('Cimahi', 0.7195360064506531), ('Bogor', 0.6814818382263184), ('Tasikmalaya', 0.6436307430267334), ('Jatinangor', 0.6386457681655884), ('Sukabumi', 0.6320213079452515), ('Garut', 0.6243470311164856), ('Cianjur', 0.6073681116104126), ('Baleendah', 0.605322539806366), ('Purwakarta', 0.6000133156776428), ('Majalaya', 0.5907973051071167)]
Kedekatan Yogyakarta-Surakarta: 0.8003842830657959
Kedekatan Yogyakarta-Semarang: 0.6383472681045532
makanan-rendang, minuman-?: [('martabak', 0.8608412014726827), ('Minuman', 0.8579015808509024), ('lemon', 0.8537830258525567), ('wajik', 0.8527022811632222), ('Sambal',